# **IBM DATA SCIENCE CAPSTONE PROJECT**
***
## **Introduction**
    Having completed the Python for data science course, the data visualization with Python course, and finally, the data analysis with Python cours it is now time to complete a capstone project.

## Project Objectives
    Say  you live on the west side of the city ofToronto in Canada. You love your neighborhood, mainly because of all the great amenities and other types of venues that exist in the neighborhood, such as gourmet fast food joints, pharmacies, parks, grad schools and so on.
    
    Now, say you receive a job offer from a great company on the other side of thecity with great career prospects. 
    However, given the far distance from your currentplace you  must move if you decide to accept the offer.
    
    Wouldn't it be great if you're able to determine neighborhoods on the other side of the city that are exactly the same as your current neighborhood or  perhaps similar neighborhoods that are at least closer to your new job? 
    
    

In [81]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto 
In this section we will use python to build the code to scrape the following Wikipedia page, [Canada Postal codes wikipage] (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![Canada Poscode dataframe](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1596412800000&hmac=RJkEkB5DDXJ82gfn_KPq-V7Ob-PgQbhW98_3BKA58k8)

---
### Part 1 - Import and Process Dataset
#### Library import

In [82]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip3 install  geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
!pip3 install  sklearn 
from sklearn.cluster import KMeans

!pip3 install folium
import folium # map rendering library

!pip3 install bs4
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

print('Libraries imported.')

Libraries imported.


#### scrap data from wikipedia

In [83]:
!pip3 install lxml
import lxml
import pandas as pd

Toronto_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0, attrs={"class":"wikitable sortable"})[0]

**RAW DATA**

In [84]:
print ("Dataset size:",Toronto_df.shape)
display(Toronto_df.head())

Dataset size: (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**PROCESS DATA**

In [85]:
#lets drop Borough that has cells with 'Not assigned' values
Not_assigned = Toronto_df[Toronto_df['Borough'] == 'Not assigned'].index
    
# Delete these row indexes from dataFrame
Toronto_df.drop(Not_assigned , inplace=True)

#Change cells that are having 'Not assigned' values in Neighborhood coloumn to equal its corresponding Borough index
Toronto_df.loc[Toronto_df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = Toronto_df['Borough']

#Group Neighborhoods having the same postalcode
Toronto_df = Toronto_df.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
Toronto_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
display(Toronto_df.head())
print('Size of the DataFrame:',Toronto_df.shape)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Size of the DataFrame: (103, 3)


---
### Part 2 - Merge Lattitude and Longitude


In [86]:
# Export dataframe to csv
Toronto_df.to_csv('Toronto.PART_1_df.csv',index=False)

# Re-import our Toronto neighborhood csv to dataframe in PART 1
Part1_data_csv = "Toronto.PART_1_df.csv"
Toronto_df = pd.read_csv(Part1_data_csv).set_index("PostalCode")
Toronto_df.rename_axis("PostalCode", axis='index', inplace=True)
#Toronto_df.head()

In [87]:
#  Retrieve the geographical coordinates of each postal code from the given link to a csv file
toronto_geo = 'https://cocl.us/Geospatial_data'
geo_data = pd.read_csv(toronto_geo).set_index("Postal Code")
geo_data.rename_axis("PostalCode", axis='index', inplace=True)
geo_data.head()

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [88]:
# Combining Two data frames with mapped postcodes
toronto_neighborhoods = Toronto_df.join(geo_data)
# Export toronto_neighborhoods to csv
toronto_neighborhoods.to_csv('Toronto.PART_2_df.csv',index=False)
toronto_neighborhoods=toronto_neighborhoods.reset_index()
display(toronto_neighborhoods.head())
print('Size of the DataFrame:',toronto_neighborhoods.shape)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Size of the DataFrame: (103, 5)


---
### Part 3 - foursquare exploration and clustering of the neighborhoods in Toronto

In [89]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### Select neigborhoods containing 'Toronto' in their names

In [90]:
display(toronto_neighborhoods.head())
toronto=toronto_neighborhoods["Borough"].unique()
print("List of Boroughs in toronto (canada):")
display(pd.DataFrame(toronto))

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


List of Boroughs in toronto (canada):


,0
0,Scarborough
1,North York
2,East York
3,East Toronto
4,Central Toronto
5,Downtown Toronto
6,York
7,West Toronto
8,Mississauga
9,Etobicoke


Boroughs containing toronto in name

In [91]:
Tor_data = toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
pd.DataFrame(Tor_data["Borough"].unique())

,0
0,East Toronto
1,Central Toronto
2,Downtown Toronto
3,West Toronto


In [92]:
Tor_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


### Initialise foursquare API

In [93]:
CLIENT_ID = '145V5MLED3MPKR54P4GP34C3UY1GB5EAU2KE5K5R5DIPGF3U' # your Foursquare ID
CLIENT_SECRET = 'G4HZPLQFHXSYOYP5XHVW3UQ1Y4XZ0LYMQWOFCNGDAKMIT2NA' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 50
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 145V5MLED3MPKR54P4GP34C3UY1GB5EAU2KE5K5R5DIPGF3U
CLIENT_SECRET:G4HZPLQFHXSYOYP5XHVW3UQ1Y4XZ0LYMQWOFCNGDAKMIT2NA


In [94]:
address = 'Central Toronto, Toronto'
#address = '12211 clearfork drive, Houston, TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude_tor = location.latitude
longitude_tor = location.longitude
print('The geograpical coordinate of Central toronto are {}, {}.'.format(latitude_tor, longitude_tor))

The geograpical coordinate of Central toronto are 43.6534817, -79.3839347.


In [95]:
map_tor = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Tor_data['Latitude'], Tor_data['Longitude'], Tor_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_tor)  
    
map_tor

### Using foursquare API get venues within 500m of 39 "Toronto" in name neigborhoods
#### Test using one neighborhood ("Beaches")

In [96]:
neighborhood_latitude = Tor_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = Tor_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = Tor_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "The Beaches" are 43.67635739999999, -79.2930312.


In [97]:
LIMIT = 100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_tor, longitude_tor, VERSION, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f25d1fca3822e313dd7ba60'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 87,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

In [98]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [99]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

<ipython-input-99-9d4491af7edd>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Poke Guys,Poke Place,43.654895,-79.385052
4,Indigo,Bookstore,43.653515,-79.380696
5,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
6,Textile Museum of Canada,Art Museum,43.654396,-79.386500
7,Old City Hall,Monument / Landmark,43.652009,-79.381744
8,CF Toronto Eaton Centre,Shopping Mall,43.654540,-79.380677
9,LUSH,Cosmetics Shop,43.653557,-79.380400


In [100]:
print('{} venues were returned by Foursquare for Beaches neighnorhood.'.format(nearby_venues.shape[0]))

87 venues were returned by Foursquare for Beaches neighnorhood.


#### Generalize to all neigborhoods containg "toronto" in name

In [101]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [102]:
Tor_venues = getNearbyVenues(names=Tor_data['Neighbourhood'],
                                   latitudes=Tor_data['Latitude'],
                                   longitudes=Tor_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [103]:
display(Tor_venues.head())
display(Tor_venues.tail())


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1629,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1630,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
1631,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
1632,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center
1633,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Revolution Recording,43.662561,-79.326940,Recording Studio


In [104]:
Tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [105]:
print('There are {} uniques categories.'.format(len(Tor_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [106]:
#one hot encoding
Tor_onehot = pd.get_dummies(Tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tor_onehot['Neighborhood'] = Tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Tor_onehot.columns[-1]] + list(Tor_onehot.columns[:-1])
Tor_onehot = Tor_onehot[fixed_columns]

print(Tor_onehot.shape)
Tor_grouped = Tor_onehot.groupby('Neighborhood').mean().reset_index()
Tor_grouped.head()

(1634, 233)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.000000,0.0,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.035088,0.0,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.00,0.0,0.035088,0.105263,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.00,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.017544,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000

#### Select Top 10 venues by neighborhood

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [108]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Tor_grouped['Neighborhood']

for ind in np.arange(Tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Burrito Place,Restaurant,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant
5,Christie,Grocery Store,Café,Park,Restaurant,Candy Store
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym
8,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop
9,Davisville North,Park,Sandwich Place,Hotel,Food & Drink Shop,Department Store


### Cluster the neighborhoods into 5 clusters using k-means based on Top10 venues

In [109]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

#Tor_data = Tor_data.drop(16)
# set number of clusters
kclusters = 5

Tor_grouped_clustering = Tor_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
len(kmeans.labels_)
Tor_data.shape

(39, 5)

In [110]:
#Add kmeans.labels_ into the dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Tor_merged = Tor_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Tor_merged = Tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Tor_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Women's Store,Deli / Bodega
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Gym,Pet Store,Brewery,Sandwich Place,Burrito Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Bakery,Gastropub,American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Women's Store,Dessert Shop
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Park,Sandwich Place,Hotel,Food & Drink Shop,Department Store
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,2,Clothing Store,Coffee Shop,Yoga Studio,Bagel Shop,Spa
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Park,Playground,Tennis Court,Women's Store,Deli / Bodega
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,2,Pub,Coffee Shop,Fried Chicken Joint,Liquor Store,Restaurant


### Display Color coded labels on a map

In [111]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_tor, longitude_tor], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tor_merged['Latitude'], Tor_merged['Longitude'], Tor_merged['Neighbourhood'], Tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Analyse the clusters to find commonalities

In [112]:
print("CLUSTER 1")
display(Tor_merged.loc[Tor_merged['Cluster Labels'] == 0, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]])
print("\nCLUSTER 2")
display(Tor_merged.loc[Tor_merged['Cluster Labels'] == 1, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]])
print("\nCLUSTER 3")
display(Tor_merged.loc[Tor_merged['Cluster Labels'] == 2, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]])
print("\nCLUSTER 4")
display(Tor_merged.loc[Tor_merged['Cluster Labels'] == 3, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]])
print("\nCLUSTER 5")
display(Tor_merged.loc[Tor_merged['Cluster Labels'] == 4, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]])


CLUSTER 1


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Women's Store,Deli / Bodega



CLUSTER 2


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Central Toronto,1,Park,Bus Line,Swim School,Women's Store,Dessert Shop
23,Central Toronto,1,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Women's Store



CLUSTER 3


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop
2,East Toronto,2,Gym,Pet Store,Brewery,Sandwich Place,Burrito Place
3,East Toronto,2,Café,Coffee Shop,Bakery,Gastropub,American Restaurant
5,Central Toronto,2,Park,Sandwich Place,Hotel,Food & Drink Shop,Department Store
6,Central Toronto,2,Clothing Store,Coffee Shop,Yoga Studio,Bagel Shop,Spa
7,Central Toronto,2,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop
9,Central Toronto,2,Pub,Coffee Shop,Fried Chicken Joint,Liquor Store,Restaurant
11,Downtown Toronto,2,Café,Restaurant,Coffee Shop,Bakery,Pizza Place
12,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar
13,Downtown Toronto,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot



CLUSTER 4


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Central Toronto,3,Park,Playground,Tennis Court,Women's Store,Deli / Bodega
10,Downtown Toronto,3,Park,Playground,Trail,Women's Store,Deli / Bodega



CLUSTER 5


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Central Toronto,4,Home Service,Garden,Women's Store,Deli / Bodega,Ethiopian Restaurant
